Import and Path

In [2]:
#import packages and set paths
import os, sys
import numpy as np 
from pathlib import Path 
import pandas as pd
import pickle
import dask
from dask import delayed
from dask.distributed import Client, progress, LocalCluster

path_to_mgr = Path('.../PoroNet')
sys.path.append(str(path_to_mgr))
import mofography as mgr

Load the MOF structures from the feather file

In [3]:
path_to_df = Path('.../Selected_MOFs_for_H2.feather')
df_Tobacco_training = pd.read_feather(path_to_df)
Tobacco_atoms = mgr.atoms_from_mofdb_df(df_Tobacco_training)

  0%|          | 0/10 [00:00<?, ?it/s]/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(
100%|██████████| 10/10 [00:01<00:00,  7.33it/s]


Create a workflow for pore graphs (using Dask to perform parallel calculation)

In [4]:
def regions_maxima_radii_from_dgrid(dgrid, mask_thickness,h, threshold_abs):
    regions, maxima = mgr.regions_from_dgrid_with_threshold_abs(dgrid, mask_thickness,h,threshold_abs)
    maxima_radii = dgrid[tuple(maxima.T )]
    print('made the regions and maxima')
    return regions, maxima, maxima_radii

def make_rag(regions, maxima, maxima_radii, dgrid, mat_atoms,spacing):
    connections = mgr.connections_from_regions_and_dgrid(regions, dgrid, maxima, mat_atoms)
    rag = mgr.rag_from_connections_pixel_multi(regions,connections, maxima, only_use_internal=False)
    rag  = mgr.add_pixel_ratio_to_rag(rag, mat_atoms, spacing) 
    shape = dgrid.shape
    rag = mgr.add_maxima_to_rag(rag, maxima, maxima_radii, shape, mat_atoms)
    rag = mgr.add_volume_to_rag(rag,mat_atoms)
    print('made the rag')
    return rag

def energy_calculation(mat_atoms, spacing, blocksize,forcefield_mixed,cutoff,probe_symbol):
    egrid = mgr.egrid_from_atoms2(mat_atoms, spacing, blocksize, forcefield_mixed, cutoff, probe_symbol, return_ncells=False, precompute_aabb=False)
    egrid= mgr.egrid_transfer(egrid)
    einterp=mgr.get_energy_interpolator(egrid)
    print('made the einterp')
    return einterp

def energy_histogram(rag,regions,einterp,pbc_groups):
    bins1=np.array([-np.inf,-10])
    bins2=np.linspace(-9,-1,9)
    bins3 = np.array([0, np.inf])
    bins=np.concatenate((bins1,bins2,bins3))
    rag=mgr.add_vdw_hist_to_rag_probability_right(rag, regions, einterp, use_pbc=True, pbc_groups=pbc_groups, bins_energy=bins)
    print('made the bins and added ehist')
    return rag

In [5]:
#delay task
delayed_rag_futures = []
for mat_atoms in Tobacco_atoms:
    dgrid = delayed(mgr.dgrid_from_atoms_cpu_no_aabb)(mat_atoms, spacing=0.5)
    regions_maxima_radii = delayed(regions_maxima_radii_from_dgrid)(dgrid, mask_thickness=0, h=0.5, threshold_abs=1)
    rag = delayed(make_rag)(regions=regions_maxima_radii[0], maxima=regions_maxima_radii[1],maxima_radii=regions_maxima_radii[2], dgrid=dgrid, mat_atoms=mat_atoms, spacing=0.5)
    pbc_groups = delayed(mgr.apply_pbc)(regions_maxima_radii[0], regions_maxima_radii[1], regions_maxima_radii[2], mat_atoms, return_conn=True)
    rag = delayed(mgr.add_pbc_to_rag)(rag,pbc_groups)
    einterp=delayed(energy_calculation)(mat_atoms,spacing=0.5, blocksize=20000, forcefield_mixed=None,cutoff=12.8,probe_symbol='H_com')
    rag = delayed(energy_histogram)(rag,regions=regions_maxima_radii[0],einterp=einterp,pbc_groups=pbc_groups)
    delayed_rag_futures.append(rag)


In [6]:
# Create a Dask client with 64 CPUs for parallel computation
client = Client(n_workers=64)

In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 488.28 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33117,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 488.28 GiB
Comm: tcp://127.0.0.1:42731,Total threads: 1
Dashboard: http://127.0.0.1:35341/status,Memory: 7.63 GiB
Nanny: tcp://127.0.0.1:37545,


In [8]:
#Start the delayed tasks 
futures = dask.persist(*delayed_rag_futures) 

In [9]:
#Check the progress
progress(futures)

VBox()

/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng3

made the regions and maxima
made the regions and maxima
made the einterp


ic| regions_to_check: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)
ic| len(outer): 12


made the regions and maxima
made the regions and maxima


...

made the rag
made the regions and maxima


ic| regions_to_check: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                             18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int32)


made the bins and added ehist


/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
ic| len(outer): 28
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


made the rag
made the bins and added ehist


In [ ]:
#Stop the task
client.shutdown()

Etraction of results

In [10]:
#Collecting results from the completed delayed task
results = client.compute(futures)
rags = [r.result() if r.status == 'finished' else None for r in results]

In [11]:
# Check how many MOFs failed (i.e., no pore graph)
np.sum([a == None for a in rags])

0

In [12]:
#Extract the pore graphs and find the indice of none
rag_list = []
none_indices = []

for i, g in enumerate(rags):
    if g is None:
        none_indices.append(i) 
    else:
        rag_list.append(g) 

In [14]:
rag_list

In [16]:
#Store the results as a pkl file
pickle.dump(rag_list, open(Path('Pore_Graph_for_H2.pkl'), 'wb'))